<a href="https://colab.research.google.com/github/Utsav147/Google-Colab-TF-Practice/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy

In [2]:
try:
    import six
    print("The 'six' library is installed.")
except ImportError:
    print("The 'six' library is not installed.")

The 'six' library is installed.


In [3]:
import tensorflow as tf
print(tf.version)

<module 'tensorflow._api.v2.version' from '/usr/local/lib/python3.12/dist-packages/tensorflow/_api/v2/version/__init__.py'>


In [ ]:
rank1_tensor=tf.Variable([[[[[[[["Test","ok"],["Test","ok"],["Test","ok"]]]]]]]],tf.string)
rank1_tensor.shape

TensorShape([1, 1, 1, 1, 1, 1, 3, 2])

In [ ]:
t=tf.zeros([5,5,5,5])
# print(t)

In [21]:
# Cell 1: Import Required Libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

print(f"TensorFlow Version: {tf.__version__}")
print(f"Keras Version: {keras.__version__}")

TensorFlow Version: 2.19.0
Keras Version: 3.10.0


In [22]:
# Cell 2: Generate Historical Weather DataFrame (Past 30 Days)
def generate_weather_data(days=30):
    """
    Generate synthetic weather data for past 30 days with realistic patterns
    Features: Temperature, Humidity, Pressure, Wind Speed, Cloud Cover, Visibility, Dew Point
    """
    np.random.seed(42)

    end_date = datetime.now()
    dates = [end_date - timedelta(days=i) for i in range(days-1, -1, -1)]

    data = []

    for i, date in enumerate(dates):
        # Create seasonal temperature pattern
        day_of_year = date.timetuple().tm_yday
        seasonal_temp = 20 + 10 * np.sin((day_of_year / 365) * 2 * np.pi)

        # Add daily variation and random noise
        temperature = seasonal_temp + np.random.normal(0, 3)

        # Generate correlated weather features
        humidity = 60 + np.random.normal(0, 15)
        humidity = np.clip(humidity, 20, 100)

        pressure = 1013 + np.random.normal(0, 10)

        wind_speed = abs(np.random.normal(12, 5))

        cloud_cover = np.random.uniform(0, 100)

        visibility = 10 + np.random.normal(0, 3)
        visibility = np.clip(visibility, 1, 15)

        dew_point = temperature - ((100 - humidity) / 5)

        data.append({
            'Date': date.strftime('%Y-%m-%d'),
            'Temperature': round(temperature, 2),
            'Humidity': round(humidity, 2),
            'Pressure': round(pressure, 2),
            'Wind_Speed': round(wind_speed, 2),
            'Cloud_Cover': round(cloud_cover, 2),
            'Visibility': round(visibility, 2),
            'Dew_Point': round(dew_point, 2)
        })

    return pd.DataFrame(data)

# Generate the dataset
df = generate_weather_data(30)

print("="*80)
print("HISTORICAL WEATHER DATA - PAST 30 DAYS")
print("="*80)
print(f"\nDataset Shape: {df.shape}")
print(f"Features: {list(df.columns)}\n")
print(df.to_string(index=False))
print("\n" + "="*80)
print("DATASET STATISTICS")
print("="*80)
print(df.describe())

HISTORICAL WEATHER DATA - PAST 30 DAYS

Dataset Shape: (30, 8)
Features: ['Date', 'Temperature', 'Humidity', 'Pressure', 'Wind_Speed', 'Cloud_Cover', 'Visibility', 'Dew_Point']

      Date  Temperature  Humidity  Pressure  Wind_Speed  Cloud_Cover  Visibility  Dew_Point
2025-10-03        11.50     57.93   1019.48       19.62        15.60       10.84       3.08
2025-10-04        13.05     51.29   1007.75        9.14        52.48        7.23       3.30
2025-10-05         4.85     51.57   1002.87       13.57        29.21        7.89      -4.83
2025-10-06         3.62     50.56   1018.98       24.80         4.65       11.18      -6.27
2025-10-07         6.60     65.64   1006.99       10.54        68.42        6.94      -0.27
2025-10-08         9.59     52.00   1012.94       10.85        66.25       11.17      -0.01
2025-10-09        10.73     30.60    999.72       12.98        96.96       10.66      -3.15
2025-10-10        12.77     44.86    997.17       15.87        35.68        8.39      

In [23]:
# Cell 3: Prepare Data for LSTM Model
# Select features for modeling
feature_columns = ['Temperature', 'Humidity', 'Pressure', 'Wind_Speed',
                   'Cloud_Cover', 'Visibility', 'Dew_Point']

data = df[feature_columns].values

# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

print("="*80)
print("DATA PREPROCESSING")
print("="*80)
print(f"Original Data Shape: {data.shape}")
print(f"Scaled Data Shape: {scaled_data.shape}")
print(f"\nFeatures Selected: {feature_columns}")
print(f"Number of Features: {len(feature_columns)}")
print(f"\nScaler Min Values:\n{scaler.data_min_}")
print(f"\nScaler Max Values:\n{scaler.data_max_}")

def create_sequences(data, seq_length=7):
    """
    Create sequences for LSTM training
    Uses past 'seq_length' days to predict next day
    """
    X, y = [], []

    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length])

    return np.array(X), np.array(y)

# Create sequences with 7-day lookback
SEQ_LENGTH = 7
X, y = create_sequences(scaled_data, SEQ_LENGTH)

print(f"\n{'='*80}")
print("SEQUENCE CREATION")
print("="*80)
print(f"Lookback Period: {SEQ_LENGTH} days")
print(f"X shape: {X.shape} (samples, timesteps, features)")
print(f"y shape: {y.shape} (samples, features)")

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

print(f"\nTraining set: {X_train.shape}")
print(f"Testing set: {X_test.shape}")
print("="*80)

DATA PREPROCESSING
Original Data Shape: (30, 7)
Scaled Data Shape: (30, 7)

Features Selected: ['Temperature', 'Humidity', 'Pressure', 'Wind_Speed', 'Cloud_Cover', 'Visibility', 'Dew_Point']
Number of Features: 7

Scaler Min Values:
[  3.62  30.6  997.17   4.68   4.08   2.03  -6.27]

Scaler Max Values:
[  16.26   92.86 1023.42   24.8    98.57   15.     11.19]

SEQUENCE CREATION
Lookback Period: 7 days
X shape: (23, 7, 7) (samples, timesteps, features)
y shape: (23, 7) (samples, features)

Training set: (18, 7, 7)
Testing set: (5, 7, 7)


In [24]:
# Cell 4: Build, Train and Evaluate LSTM Model
def build_lstm_model(input_shape):
    """
    Build LSTM model for multi-feature weather forecasting
    Architecture: LSTM layers with dropout for regularization
    """
    model = Sequential([
        # First LSTM layer with return sequences
        LSTM(64, return_sequences=True, input_shape=input_shape),
        Dropout(0.2),

        # Second LSTM layer
        LSTM(32, return_sequences=False),
        Dropout(0.2),

        # Dense layers
        Dense(16, activation='relu'),
        Dense(input_shape[1])  # Output layer with same features as input
    ])

    # Compile model
    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='mse',
        metrics=['mae']
    )

    return model

# Build the model
model = build_lstm_model((SEQ_LENGTH, len(feature_columns)))

print("="*80)
print("LSTM MODEL ARCHITECTURE")
print("="*80)
model.summary()
print("="*80)

# Train the model
print("\nTraining the LSTM model...\n")
print("="*80)

history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=4,
    validation_data=(X_test, y_test),
    verbose=1
)

print("\n" + "="*80)
print("TRAINING COMPLETED!")
print("="*80)

# Evaluate on test set
test_loss, test_mae = model.evaluate(X_test, y_test, verbose=0)

print("\n" + "="*80)
print("MODEL EVALUATION METRICS")
print("="*80)
print(f"Test Loss (MSE): {test_loss:.6f}")
print(f"Test MAE: {test_mae:.6f}")

# Make predictions on test set
test_predictions_scaled = model.predict(X_test, verbose=0)
test_predictions = scaler.inverse_transform(test_predictions_scaled)
y_test_actual = scaler.inverse_transform(y_test)

# Calculate MAE for each feature
print(f"\n{'='*80}")
print("MEAN ABSOLUTE ERROR BY FEATURE")
print("="*80)
for i, feature in enumerate(feature_columns):
    mae = np.mean(np.abs(test_predictions[:, i] - y_test_actual[:, i]))
    print(f"{feature:15s}: {mae:.4f}")

print("="*80)

LSTM MODEL ARCHITECTURE


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 7, 64)          │        18,432 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 7, 64)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 32)             │        12,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 7)              │           119 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 31,495 (123.03 KB)

 Trainable params: 31,495 (123.03 KB)

 Non-trainable params: 0 (0.00 B)


Training the LSTM model...

Epoch 1/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 162ms/step - loss: 0.3484 - mae: 0.5234 - val_loss: 0.3136 - val_mae: 0.4930
Epoch 2/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - loss: 0.2576 - mae: 0.4446 - val_loss: 0.2114 - val_mae: 0.4002
Epoch 3/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.1680 - mae: 0.3452 - val_loss: 0.1262 - val_mae: 0.3134
Epoch 4/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - loss: 0.0959 - mae: 0.2546 - val_loss: 0.0749 - val_mae: 0.2287
Epoch 5/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - loss: 0.0970 - mae: 0.2462 - val_loss: 0.0613 - val_mae: 0.2103
Epoch 6/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.1073 - mae: 0.2711 - val_loss: 0.0622 - val_mae: 0.2091
Epoch 7/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - loss: 0.0840 - mae: 0.2335 - val_loss: 0.0668 - val_mae: 0.2282
Epoch 8/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 0.0754 - mae: 0.2157 - val_loss: 0.0679 - val_mae: 0.2371
Epoch 9/100
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 42

In [25]:
# Cell 5: Predict Today's Weather
# Use the last 7 days to predict today
last_sequence = scaled_data[-SEQ_LENGTH:]
last_sequence_reshaped = last_sequence.reshape(1, SEQ_LENGTH, len(feature_columns))

# Predict today's weather
today_prediction_scaled = model.predict(last_sequence_reshaped, verbose=0)
today_prediction = scaler.inverse_transform(today_prediction_scaled)

print("="*80)
print("TODAY'S WEATHER PREDICTION")
print("="*80)
print(f"Prediction Date: {datetime.now().strftime('%Y-%m-%d')}\n")

for i, feature in enumerate(feature_columns):
    unit = "°C" if feature == "Temperature" or feature == "Dew_Point" else \
           "%" if feature == "Humidity" or feature == "Cloud_Cover" else \
           "hPa" if feature == "Pressure" else \
           "km/h" if feature == "Wind_Speed" else "km"
    print(f"{feature:15s}: {today_prediction[0][i]:8.2f} {unit}")

print("="*80)

TODAY'S WEATHER PREDICTION
Prediction Date: 2025-11-01

Temperature    :    10.27 °C
Humidity       :    54.25 %
Pressure       :  1008.14 hPa
Wind_Speed     :    10.07 km/h
Cloud_Cover    :    66.29 %
Visibility     :     8.40 km
Dew_Point      :     1.83 °C


In [26]:
# Cell 6: Predict Next 7 Days Weather
def predict_next_days(model, last_sequence, scaler, days=7):
    """
    Predict weather for the next 'days' days
    Uses iterative prediction: each prediction becomes input for next
    """
    predictions = []
    current_sequence = last_sequence.copy()

    for i in range(days):
        # Reshape for prediction
        current_input = current_sequence.reshape(1, SEQ_LENGTH, len(feature_columns))

        # Predict next day
        next_pred_scaled = model.predict(current_input, verbose=0)
        next_pred = scaler.inverse_transform(next_pred_scaled)

        predictions.append(next_pred[0])

        # Update sequence: remove oldest, add newest
        current_sequence = np.vstack([current_sequence[1:], next_pred_scaled[0]])

    return np.array(predictions)

# Predict next 7 days
print("="*80)
print("GENERATING 7-DAY FORECAST...")
print("="*80)

future_predictions = predict_next_days(model, scaled_data[-SEQ_LENGTH:], scaler, days=7)

# Create dataframe for predictions
future_dates = [(datetime.now() + timedelta(days=i+1)).strftime('%Y-%m-%d') for i in range(7)]
predictions_df = pd.DataFrame(future_predictions, columns=feature_columns)
predictions_df.insert(0, 'Date', future_dates)

print("\n" + "="*80)
print("NEXT 7 DAYS WEATHER FORECAST")
print("="*80)
print(predictions_df.to_string(index=False))
print("\n" + "="*80)

GENERATING 7-DAY FORECAST...

NEXT 7 DAYS WEATHER FORECAST
      Date  Temperature  Humidity    Pressure  Wind_Speed  Cloud_Cover  Visibility  Dew_Point
2025-11-02    10.274011 54.246628 1008.139648   10.065694    66.290527    8.396559   1.833741
2025-11-03    10.674711 58.760124 1009.852661   10.437902    65.382919    9.156222   2.545881
2025-11-04    10.808354 56.348705 1008.940308   10.418057    73.231812    8.646022   2.450877
2025-11-05    10.141576 58.774776 1009.012024    9.635486    65.555328    9.391857   2.121711
2025-11-06    10.275707 57.153996 1008.829346    9.913975    64.260170    9.078058   2.106883
2025-11-07    10.266674 57.352142 1008.919800    9.996714    60.194740    9.002041   2.057552
2025-11-08     9.877362 59.339069 1008.579712    9.240508    62.422386    9.610801   2.019758



In [27]:
# Cell 7: Summary and Model Information
print("="*80)
print("WEATHER FORECASTING MODEL SUMMARY")
print("="*80)
print(f"\nModel Type: LSTM (Long Short-Term Memory) Neural Network")
print(f"Architecture: 2 LSTM layers (64, 32 units) + Dense layers")
print(f"Sequence Length: {SEQ_LENGTH} days lookback")
print(f"Features Used: {len(feature_columns)}")
print(f"  - {', '.join(feature_columns)}")
print(f"\nTraining Data: {len(df)} days of historical weather data")
print(f"Training Epochs: 100")
print(f"Batch Size: 4")
print(f"Optimizer: Adam (learning rate: 0.001)")
print(f"Loss Function: Mean Squared Error (MSE)")
print(f"Evaluation Metric: Mean Absolute Error (MAE)")
print(f"\nPredictions Generated:")
print(f"  ✓ Today's weather prediction")
print(f"  ✓ Next 7 days forecast")
print("\n" + "="*80)
print("WHY LSTM FOR WEATHER FORECASTING?")
print("="*80)
print("✓ Excellent for time series data and sequential patterns")
print("✓ Captures temporal dependencies in weather patterns")
print("✓ Handles multiple features simultaneously (7 features)")
print("✓ Remembers long-term patterns through cell states")
print("✓ Can learn complex non-linear relationships")
print("✓ Suitable for multi-step ahead forecasting")
print("✓ Resistant to vanishing gradient problem")
print("="*80)

# Display comparison: Last 3 days historical vs Next 3 days predicted
print("\n" + "="*80)
print("TEMPERATURE TREND COMPARISON")
print("="*80)
print("\nLast 3 Days (Historical):")
print(df[['Date', 'Temperature']].tail(3).to_string(index=False))
print("\nNext 3 Days (Predicted):")
print(predictions_df[['Date', 'Temperature']].head(3).to_string(index=False))
print("="*80)

WEATHER FORECASTING MODEL SUMMARY

Model Type: LSTM (Long Short-Term Memory) Neural Network
Architecture: 2 LSTM layers (64, 32 units) + Dense layers
Sequence Length: 7 days lookback
Features Used: 7
  - Temperature, Humidity, Pressure, Wind_Speed, Cloud_Cover, Visibility, Dew_Point

Training Data: 30 days of historical weather data
Training Epochs: 100
Batch Size: 4
Optimizer: Adam (learning rate: 0.001)
Loss Function: Mean Squared Error (MSE)
Evaluation Metric: Mean Absolute Error (MAE)

Predictions Generated:
  ✓ Today's weather prediction
  ✓ Next 7 days forecast

WHY LSTM FOR WEATHER FORECASTING?
✓ Excellent for time series data and sequential patterns
✓ Captures temporal dependencies in weather patterns
✓ Handles multiple features simultaneously (7 features)
✓ Remembers long-term patterns through cell states
✓ Can learn complex non-linear relationships
✓ Suitable for multi-step ahead forecasting
✓ Resistant to vanishing gradient problem

TEMPERATURE TREND COMPARISON

Last 3 Days 

In [ ]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

# Load dataset
df = pd.read_csv("weather.csv")

# Create lag features
def create_lag_features(data, col, lags=3):
    for i in range(1, lags + 1):
        data[f"{col}_t-{i}"] = data[col].shift(i)
    return data

# Apply lagging on important features
for feature in ['temp', 'humidity', 'pressure', 'wind_speed']:
    df = create_lag_features(df, feature, lags=3)

# Drop rows having NaN after lag
df = df.dropna()

# Select features and target
X = df.drop(columns=['date', 'target_temp_next_day'])
y = df['target_temp_next_day']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Build XGBoost model
model = XGBRegressor(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=5,
    subsample=0.8,
    colsample_bytree=0.8
)

model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print("MAE:", mae)

# Predict next day's temperature
latest_row = X.tail(1)
next_temp = model.predict(latest_row)
print("Predicted next day temperature:", next_temp[0])
